In [1]:
import json
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
import pickle

Using TensorFlow backend.


In [2]:
from __future__ import print_function
from keras import backend as K

In [3]:
def parse(file):
    for l in open(file, 'r'):
        yield json.loads(l)

In [4]:
data = parse('news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json')

In [5]:
df = pd.DataFrame(data)

In [6]:
df.values

array([['https://www.theonion.com/thirtysomething-scientists-unveil-doomsday-clock-of-hai-1819586205',
        'thirtysomething scientists unveil doomsday clock of hair loss',
        1],
       ['https://www.huffingtonpost.com/entry/donna-edwards-inequality_us_57455f7fe4b055bb1170b207',
        'dem rep. totally nails why congress is falling short on gender, racial equality',
        0],
       ['https://www.huffingtonpost.com/entry/eat-your-veggies-9-delici_b_8899742.html',
        'eat your veggies: 9 deliciously different recipes', 0],
       ...,
       ['https://www.huffingtonpost.com/entry/andrew-ahn-independent-spirit-awards_us_58b44741e4b060480e0a2c30',
        'the most beautiful acceptance speech this week came from a queer korean',
        0],
       ['https://www.theonion.com/mars-probe-destroyed-by-orbiting-spielberg-gates-space-1819564363',
        'mars probe destroyed by orbiting spielberg-gates space palace',
        1],
       ['https://www.theonion.com/dad-clarifies

In [7]:
df = df.drop(df.columns[0], axis=1)

In [8]:
target = df.is_sarcastic
df = df.headline

In [9]:
headlines = df.values
y = target.values
headlines

array(['thirtysomething scientists unveil doomsday clock of hair loss',
       'dem rep. totally nails why congress is falling short on gender, racial equality',
       'eat your veggies: 9 deliciously different recipes', ...,
       'the most beautiful acceptance speech this week came from a queer korean',
       'mars probe destroyed by orbiting spielberg-gates space palace',
       'dad clarifies this not a food stop'], dtype=object)

In [10]:
headlines_train, headlines_test, y_train, y_test = train_test_split(headlines, y, test_size=0.25, random_state=1000)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(headlines_train)

X_train = vectorizer.transform(headlines_train)
X_test  = vectorizer.transform(headlines_test)
X_train[1]

<1x23214 sparse matrix of type '<class 'numpy.int64'>'
	with 13 stored elements in Compressed Sparse Row format>

In [12]:
input_dim = X_train.shape[1]  # Number of features
input_dim

23214

In [13]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(headlines_train)

X_train = tokenizer.texts_to_sequences(headlines_train)
X_test = tokenizer.texts_to_sequences(headlines_test)

vocab_size = len(tokenizer.word_index) + 1

print(headlines_train[2])
print(X_train[2])

calle 13 explores the power of a kiss
[13585, 893, 4532, 3, 345, 2, 6, 3310]


In [14]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
from keras.preprocessing.sequence import pad_sequences
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
print(X_train[0, :])

[   57  9489  1450  3667 13582     1  4048 13583     1  1833  4049  9490
     4  9491     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]


In [16]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [17]:
embedding_dim = 100
embedding_matrix = create_embedding_matrix(
    'glove.6B.100d.txt',
    tokenizer.word_index, embedding_dim)

In [18]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

0.8177338062348329

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

In [20]:
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(layers.Bidirectional(LSTM(25, return_sequences=True)))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

W0923 01:11:39.187860 139835701995264 deprecation_wrapper.py:119] From /home/matthew/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0923 01:11:39.209828 139835701995264 deprecation_wrapper.py:119] From /home/matthew/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0923 01:11:39.212067 139835701995264 deprecation_wrapper.py:119] From /home/matthew/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0923 01:11:39.222823 139835701995264 deprecation_wrapper.py:119] From /home/matthew/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_defaul

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          2678500   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 50)           25200     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 2,704,221
Trainable params: 25,721
Non-trainable params: 2,678,500
_________________________________________________________________


In [21]:
model.fit(X_train, y_train,
                    epochs=10,
                    validation_data=(X_test, y_test),
                    batch_size=20)

Train on 21464 samples, validate on 7155 samples
Epoch 1/10
21464/21464 [==============================] - 79s 4ms/step - loss: 0.4800 - acc: 0.7631 - val_loss: 0.3960 - val_acc: 0.8222
Epoch 2/10
21464/21464 [==============================] - 78s 4ms/step - loss: 0.3508 - acc: 0.8445 - val_loss: 0.3531 - val_acc: 0.8397
Epoch 3/10
21464/21464 [==============================] - 73s 3ms/step - loss: 0.3031 - acc: 0.8689 - val_loss: 0.3338 - val_acc: 0.8545
Epoch 4/10
21464/21464 [==============================] - 74s 3ms/step - loss: 0.2650 - acc: 0.8906 - val_loss: 0.3199 - val_acc: 0.8623
Epoch 5/10
21464/21464 [==============================] - 83s 4ms/step - loss: 0.2354 - acc: 0.9041 - val_loss: 0.3212 - val_acc: 0.8622
Epoch 6/10
21464/21464 [==============================] - 94s 4ms/step - loss: 0.2041 - acc: 0.9191 - val_loss: 0.3320 - val_acc: 0.8621
Epoch 7/10
21464/21464 [==============================] - 74s 3ms/step - loss: 0.1746 - acc: 0.9324 - val_loss: 0.3430 - val_acc:

In [ ]:
best_val_score = 0.8742

In [25]:
with open('model_bidirectional_lstm', 'wb') as f:
    pickle.dump(model, f)

In [26]:
with open('model_bidirectional_lstm', 'rb') as f:
    mp = pickle.load(f)

In [27]:
model_json = model.to_json()

In [28]:
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

In [26]:
model.save_weights('model_bidirectional_lstm_weights.h5')